In [1]:
from numpy import *
from pylab import *
import string

from datetime import date

import AgilentSCPI
import time
import numpy as np
import sys
import matplotlib.pyplot as plt
import time

In [2]:
class timedata:

    def __init__(self, name):
        self.name = name
        self.time = np.array    # creates a new empty list
        self.data = np.array    # creates a new empty list


In [3]:
def readlabdat(fname,dataset):
# read in data from the scope and make a matrix of values from it
    
    
    f = open(fname,'r')
    #header1=f.readline()
    #header2=f.readline()
    tmp1=[]
    tmp2=[]
    
    for line in f:
        columns = line.split(',')
        tmp1.append(float(columns[0]))
        tmp2.append(float(columns[1]))

    dataset.time=np.asarray(tmp1)
    dataset.data=np.asarray(tmp2)
    f.close()
        

In [4]:
# read data from oscilloscope and save to file
def saveWave(filename,chs,npts='MAXIMUM'):
    for ch in chs:
        p1,x1 = osc.OscRead2Numpy(ch,npts)
#        print np.shape(x1)
        print("channel is"+str(ch))
        if 'data' in locals():
#            print "inside if"
            data = vstack((data,x1))
        else:
            data = x1
            
    
    t = arange(len(x1))*p1[4] + p1[5]
    data = vstack((t,data)).T
    savetxt(filename, data, fmt="%12.6G", delimiter=',')
    print(shape(data)+"data points saved to "+filename)

In [5]:
# read data from oscilloscope and save to file
def saveWave(filename,chs,npts):
    for ch in chs:
        p1,x1 = osc.OscRead2Numpy(ch,npts)
#        print np.shape(x1)
        print("channel is"+str(ch))
        if 'data' in locals():
#            print "inside if"
            data = vstack((data,x1))
        else:
            data = x1
            
    
    t = arange(len(x1))*p1[4] + p1[5]
    data = vstack((t,data)).T
    savetxt(filename, data, fmt="%12.6G", delimiter=',')
    print(shape(data)+"data points saved to "+filename)

In [ ]:
osc = AgilentSCPI.SCPI("134.153.36.86")
q2=osc.query('*IDN?\n')
print q2
osc.s.send(":TIM:REF LEFT\n") 
osc.s.send(":TIM:SCAL 60us\n")
osc.s.send(":TIM:DEL 0us\n") 
osc.s.send("CHANnel2:SCALe 0.03\n")  


In [17]:
import serial

#initialization and open the port
#possible timeout values:
#    1. None: wait forever, block call
#    2. 0: non-blocking mode, return immediately
#    3. x, x is bigger than 0, float allowed, timeout block call

ser = serial.Serial()

#ser.port = "COM3"
#ser.port = "/dev/ttyS2"
ser.port="/dev/tty.usbserial-AL030TI8"

ser.baudrate = 9600
ser.bytesize = serial.EIGHTBITS		#number of bits per bytes
ser.parity = serial.PARITY_NONE		#set parity check: no parity
ser.stopbits = serial.STOPBITS_ONE	#number of stop bits

#ser.timeout = None		#block read
ser.timeout = 1			#non-block read
#ser.timeout = 2		        #timeout block read

ser.xonxoff = False		#disable software flow control
ser.rtscts = False		#disable hardware (RTS/CTS) flow control
ser.dsrdtr = False		#disable hardware (DSR/DTR) flow control
ser.writeTimeout = 2	        #timeout for write


In [18]:
fname_root = "/Users/amalcolm/Dropbox/Nonlinear/Experiments/CODE/LABCodes/STAGE"
npts=2000
StepsPerMM = 400 / (25.4 * 0.1)
dR = 1
Rm = -30
Rp = 30
R=arange(Rm,Rp+dR,dR,dtype=float)
nR = len(R)
dRSteps = int(round(StepsPerMM*dR))
RmSteps = int(round(StepsPerMM*Rm))

In [19]:
StepsPerMM = 400 / (25.4 * 0.1)
dR = 1
Rm2 = -25
Rp2 = 25
R2=arange(Rm2,Rp2+dR,dR,dtype=float)
nR2 = len(R)
dRSteps2 = int(round(StepsPerMM*dR))
RmSteps2 = int(round(StepsPerMM*Rm2))
npts=2000

In [91]:
ser.close()
ser.open()
for ii in range(2):
    ser.write("F,C,I1M-300,R".encode())
    ser.write("X".encode())
    pos=ser.readline()
    print(pos.decode())
    ser.write("Y".encode())
    pos=ser.readline()
    print(pos.decode())
    ii=ii+1

^0010501
-0000353
^0010801
-0000353


In [80]:
#ser.write('C,N,R'.encode())
#ser.write("F,C,I2M-300,R".encode())  #I2M moves up(add -)/down(no -)
#ser.write("F,C,I1M300,R".encode())  #I1M moves left(add -)/right (no -)
ser.readline();
ser.write("I1M".encode())   # query for position
pos = ser.readline()
print(pos.decode())
ser.write("F,C,I1M-300,R".encode())  #I1M moves left(add -)/right (no -)
pos = ser.readline();
print(pos.decode())
ser.write("I1M,R".encode())
pos=ser.readline()
print(pos.decode())

?252
^



In [62]:
ser.close()
# define home positions
try:
    ser.open()
except Exception:
    print("error open serial port: " + str(Exception))
    sys.exit()
if ser.isOpen():
    try:
        print("flushing input")
        ser.flushInput()
        print("flushing output")
        ser.flushOutput()
        print("sending message")
        ser.write("F,C,I1M3000,R".encode())
        time.sleep(1)
    except Exception:
        print("error communicating...: " + str(Exception))
else:
    print("cannot open serial port ")
ser.close() 

flushing input
flushing output
sending message


In [46]:
ser.open()
for istep in range(nR):
    time.sleep(60)
    osc.s.send(":TIM:REF LEFT\n") 
    osc.s.send(":TIM:SCAL 60us\n")
    osc.s.send(":TIM:DEL 0us\n") 
    osc.s.send("CHANnel2:SCALe 0.03\n")
    fname=fname_root+"K"+str(istep)+".csv"
    saveWave(fname,[2])
    ser.write('F,C,I1M'+str(dRSteps)+',R')
    time.sleep(5)
    print istep 

(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K0.csv
0
(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K1.csv
1
(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K2.csv
2
(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K3.csv
3
(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K4.csv
4
(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K5.csv
5
(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K6.csv
6
(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K7.csv
7
(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K8.csv
8
(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K9.csv
9
(7500L,)
channel is2
(7500L, 2L) data points saved to /users/Kamal_PC/Experiment/K10.csv
10
(7500L,)